In [ ]:
from IPython.display import clear_output

In [ ]:
!pip install torch 
!pip install torchvision
!pip install transformers
!pip install peft==0.10.0
!pip install datasets
!pip install pillow
!pip install underthesea
!pip install huggingface_hub
!pip install hf_xet
!pip install bert-score

clear_output()

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from peft import get_peft_model, LoraConfig, TaskType

from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
from datasets import load_dataset
from torch.utils.data import DataLoader, IterableDataset

from PIL import ImageFile
import os
from tqdm import tqdm
from huggingface_hub import hf_hub_download, HfApi

ImageFile.LOAD_TRUNCATED_IMAGES = True

In [ ]:
# Define project-specific variables
PROJECT_NAME = "ViInfographicsVQA"  # Name of the project
USERNAME = "Namronaldo2004"          # Hugging Face username
HUGGINGFACE_HUB_REPO = USERNAME + "/" + PROJECT_NAME  # Full repository name on Hugging Face Hub
BASELINE_NAME = "Flow3-modified/Text"
CHECKPOINT_FILENAME = f"{BASELINE_NAME}/latest_checkpoint.pth"
OCR_VAL_FILENAME = "final_val_OCR.txt"
OCR_TEST_FILENAME = "final_test_OCR.txt"

In [ ]:
api = HfApi()
CHECKPOINT_PATH = "./checkpoints/latest_checkpoint.pth"
if api.file_exists(
    repo_id = HUGGINGFACE_HUB_REPO, 
    filename = CHECKPOINT_FILENAME, 
    repo_type = "model"
):
    CHECKPOINT_PATH = hf_hub_download(
        repo_id = HUGGINGFACE_HUB_REPO, 
        filename = CHECKPOINT_FILENAME, 
        local_dir = "./checkpoints",  # Store the checkpoint locally in the "checkpoints" directory
        local_dir_use_symlinks = False  # Avoid using symlinks for compatibility
    )

os.makedirs("checkpoints", exist_ok = True)
print(CHECKPOINT_PATH)

In [ ]:
class ViInfographicsVQATextDataset(IterableDataset):
    """
    Custom dataset for streaming text-image pairs from a Hugging Face dataset.
    This dataset yields image-text pairs one by one, without preloading everything into memory.
    """

    def __init__(self, hf_dataset, ocr_contents, max_instances = None):
        """
        Initialize the dataset.
        
        Args:
            hf_dataset: The Hugging Face dataset containing image-text pairs.
            transform: Optional image transformations (e.g., resizing, normalization).
            max_instances: Maximum number of instances to process.
        """
        self.hf_dataset = hf_dataset
        self.ocr_contents = ocr_contents
        self.max_instances = max_instances

    def __iter__(self):
        """
        Iterator to stream dataset samples.
        
        Yields:
            A tuple (image, text) where:
            - image: Transformed image tensor
            - text: Corresponding textual prompt
        """
        count = 0
        temp_count = 0
        for sample in self.hf_dataset:
            if (self.max_instances is not None and count >= self.max_instances):
                break  # Stop if max instances reached

            question_type = sample["type"]
            if (question_type.lower() == "text"):
                ocr_info = self.ocr_contents[str(temp_count)]
                question = sample["question"]
                answer = sample["answer"]
                
                yield ocr_info, question, answer
                count += 1  # Increment the counter

            temp_count += 1

In [ ]:
class BARTPho(nn.Module):
    def __init__(
        self,
        model_name="vinai/bartpho-syllable",
        device="cpu",
        max_length=50,
        use_lora=False,
        lora_r=8,
        lora_alpha=16,
        lora_dropout=0.05,
        target_modules=["q_proj", "v_proj"]
    ):
        super().__init__()
        self.tokenizer = AutoTokenizer.from_pretrained(model_name)
        self.device = device
        self.max_length = max_length

        # Load base model
        base_model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

        # Apply LoRA if needed
        if use_lora:
            lora_config = LoraConfig(
                r=lora_r,
                lora_alpha=lora_alpha,
                lora_dropout=lora_dropout,
                bias="none",
                task_type=TaskType.SEQ_2_SEQ_LM,
                target_modules=target_modules
            )
            base_model = get_peft_model(base_model, lora_config)

        # Save full model
        self.encoder = base_model.base_model.model.model.encoder.to(device)
        self.decoder = base_model.base_model.model.model.decoder  # BART/M-BART decoder
        self.lm_head = base_model.base_model.model.lm_head

    def encode(self, input_texts):
        batch_encoder_hidden_states = []
        batch_attention_masks = []
        batch_input_ids = []
    
        for text in input_texts:
            # Split sentence into chunks (each chunk has at most 512 words)
            words = text.split()
            chunks = [' '.join(words[i * 384: (i + 1) * 384]) for i in range((len(words) + 383) // 384)]
    
            chunk_input_ids_list = []
            chunk_attention_mask_list = []
            chunk_hidden_states_list = []
    
            for chunk in chunks:
                encoded = self.tokenizer(
                    chunk,
                    return_tensors="pt",
                    padding=True,
                    truncation=True
                ).to(self.device)
    
                input_ids = encoded["input_ids"]           # shape: (1, seq_len)
                attention_mask = encoded["attention_mask"] # shape: (1, seq_len)
    
                outputs = self.encoder(
                    input_ids=input_ids,
                    attention_mask=attention_mask
                )
    
                chunk_input_ids_list.append(input_ids)
                chunk_attention_mask_list.append(attention_mask)
                chunk_hidden_states_list.append(outputs.last_hidden_state)
    
            # Concatenate all chunks → shape: (1, total_seq_len, hidden_size)
            full_input_ids = torch.cat(chunk_input_ids_list, dim=1).squeeze(0)          # shape: (total_seq_len)
            full_attention_mask = torch.cat(chunk_attention_mask_list, dim=1).squeeze(0)# shape: (total_seq_len)
            full_hidden_states = torch.cat(chunk_hidden_states_list, dim=1).squeeze(0)  # shape: (total_seq_len, hidden_size)
    
            batch_input_ids.append(full_input_ids)
            batch_attention_masks.append(full_attention_mask)
            batch_encoder_hidden_states.append(full_hidden_states)
    
        # Pad sequences in the batch to the same length
        max_seq_len = max(x.shape[0] for x in batch_input_ids)
    
        def pad_tensor(tensor, max_len, pad_value=0):
            pad_len = max_len - tensor.shape[0]
            if pad_len == 0:
                return tensor
            pad = torch.full((pad_len,), pad_value, dtype=tensor.dtype, device=tensor.device)
            return torch.cat([tensor, pad], dim=0)
    
        def pad_hidden_states(tensor, max_len):
            pad_len = max_len - tensor.shape[0]
            if pad_len == 0:
                return tensor
            pad = torch.zeros((pad_len, tensor.shape[1]), device=tensor.device)
            return torch.cat([tensor, pad], dim=0)
    
        # Stack padded tensors
        input_ids = torch.stack([pad_tensor(x, max_seq_len, self.tokenizer.pad_token_id) for x in batch_input_ids], dim=0)
        attention_mask = torch.stack([pad_tensor(x, max_seq_len, 0) for x in batch_attention_masks], dim=0)
        encoder_hidden_states = torch.stack([pad_hidden_states(x, max_seq_len) for x in batch_encoder_hidden_states], dim=0)
    
        return {
            "encoder_hidden_states": encoder_hidden_states,  # shape: (batch_size, seq_len, hidden_size)
            "attention_mask": attention_mask,                # shape: (batch_size, seq_len)
            "input_ids": input_ids                           # shape: (batch_size, seq_len)
        }

    def decode(
        self,
        answer_input_ids,
        answer_attention_mask,
        encoder_hidden_states,
        encoder_attention_mask,
    ):    
        decoder_outputs = self.decoder(
            input_ids=answer_input_ids,
            attention_mask=answer_attention_mask,
            encoder_hidden_states=encoder_hidden_states,
            encoder_attention_mask=encoder_attention_mask,
        )
        
        logits = self.lm_head(decoder_outputs.last_hidden_state)
        return logits

    def generate(self, encoder_hidden_states, encoder_attention_mask):
        batch_size = encoder_hidden_states.size(0)
        device = encoder_hidden_states.device
    
        # Bắt đầu với decoder_input_ids là eos_token_id cho mỗi dòng trong batch
        decoder_input_ids = torch.full(
            (batch_size, 1),
            fill_value=self.tokenizer.eos_token_id,
            dtype=torch.long,
            device=device
        )
    
        with torch.no_grad():
            for _ in range(self.max_length):
                logits = self.decode(
                    answer_input_ids=decoder_input_ids,
                    answer_attention_mask=torch.ones_like(decoder_input_ids, device=device),
                    encoder_hidden_states=encoder_hidden_states,
                    encoder_attention_mask=encoder_attention_mask
                )
    
                next_token = logits[:, -1, :].argmax(-1, keepdim=True)  # (B, 1)
                decoder_input_ids = torch.cat([decoder_input_ids, next_token], dim=-1)
    
                # Dừng sớm nếu tất cả dòng đều sinh ra <eos>
                if (next_token == self.tokenizer.eos_token_id).all():
                    break
    
        return self.tokenizer.batch_decode(decoder_input_ids, skip_special_tokens = True)

    def freeze_encoder(self, layers_to_freeze=None):
        if layers_to_freeze is None:
            for param in self.encoder.parameters():
                param.requires_grad = False
        else:
            for idx, layer in enumerate(self.encoder.layers):
                if idx in layers_to_freeze:
                    for param in layer.parameters():
                        param.requires_grad = False

    def unfreeze_encoder(self):
        for param in self.encoder.parameters():
            param.requires_grad = True

In [ ]:
class PositionWiseFeedForward(nn.Module):
    def __init__(self, d_model = 768, d_ff = 2048, dropout = 0.1):
        super(PositionWiseFeedForward, self).__init__()
        self.fc1 = nn.Linear(d_model, d_ff)
        self.fc2 = nn.Linear(d_ff, d_model)
        self.dropout1 = nn.Dropout(dropout)
        self.dropout2 = nn.Dropout(dropout)
        self.norm = nn.LayerNorm(d_model)

    def forward(self, x):
        out = self.dropout1(F.gelu(self.fc1(x)))
        out = self.dropout2(self.fc2(out))
        return self.norm(x + out)

class MultiHeadAttention(nn.Module):
    def __init__(self, d_model = 768, num_heads = 8, dropout = 0.1):
        super(MultiHeadAttention, self).__init__()
        self.attn = nn.MultiheadAttention(embed_dim = d_model, num_heads = num_heads, dropout = dropout, batch_first = True)
        self.norm = nn.LayerNorm(d_model)
        self.dropout = nn.Dropout(dropout)

    def forward(self, queries, keys, values, attention_mask=None):
        if attention_mask is not None:
            attention_mask = ~attention_mask.bool()  # Convert to padding mask: True = MASK
        attn_output, _ = self.attn(queries, keys, values, key_padding_mask = attention_mask)
        out = self.dropout(attn_output)
        return self.norm(queries + out)

class EncoderLayer(nn.Module):
    def __init__(self, d_model = 768, num_heads = 8, d_ff = 2048, dropout = 0.1):
        super(EncoderLayer, self).__init__()
        self.mhatt = MultiHeadAttention(d_model, num_heads, dropout)
        self.pwff = PositionWiseFeedForward(d_model, d_ff, dropout)

    def forward(self, queries, keys, values, attention_mask = None):
        att = self.mhatt(queries, keys, values, attention_mask)
        ff = self.pwff(att)
        return ff

class BiDirectionalCrossAttention(nn.Module):
    def __init__(self, d_model = 1024, num_heads = 8, d_ff = 2048, dropout = 0.1, num_layers = 3, max_len = 6000):
        super(BiDirectionalCrossAttention, self).__init__()

        self.vision_pos_embed = nn.Embedding(max_len, d_model)
        self.text_pos_embed = nn.Embedding(max_len, d_model)

        self.vision_norm = nn.LayerNorm(d_model)
        self.text_norm = nn.LayerNorm(d_model)

        self.d_model = d_model  # D = 1024

        self.vision_language_attn_layers = nn.ModuleList(
            [EncoderLayer(d_model, num_heads, d_ff, dropout) for _ in range(num_layers)]
        )
        self.language_vision_attn_layers = nn.ModuleList(
            [EncoderLayer(d_model, num_heads, d_ff, dropout) for _ in range(num_layers)]
        )
        self.vision_self_attn_layers = nn.ModuleList(
            [EncoderLayer(d_model, num_heads, d_ff, dropout) for _ in range(num_layers)]
        )
        self.language_self_attn_layers = nn.ModuleList(
            [EncoderLayer(d_model, num_heads, d_ff, dropout) for _ in range(num_layers)]
        )

    def forward(self, vision_feats, vision_mask, text_feats, text_mask):
        batch_size, v_len, _ = vision_feats.size()
        _, t_len, _ = text_feats.size()

        v_pos_ids = torch.arange(v_len, device=vision_feats.device).unsqueeze(0).repeat(batch_size, 1)
        t_pos_ids = torch.arange(t_len, device=text_feats.device).unsqueeze(0).repeat(batch_size, 1)

        vision_feats = self.vision_norm(vision_feats + self.vision_pos_embed(v_pos_ids))
        text_feats = self.text_norm(text_feats + self.text_pos_embed(t_pos_ids))

        for vl_attn, lv_attn, v_self, l_self in zip(
            self.vision_language_attn_layers,
            self.language_vision_attn_layers,
            self.vision_self_attn_layers,
            self.language_self_attn_layers
        ):
            vision_feats = vl_attn(vision_feats, text_feats, text_feats, text_mask)
            text_feats = lv_attn(text_feats, vision_feats, vision_feats, vision_mask)

            vision_feats = v_self(vision_feats, vision_feats, vision_feats, vision_mask)
            text_feats = l_self(text_feats, text_feats, text_feats, text_mask)

        fused_feats = torch.cat([vision_feats, text_feats], dim=1)  # shape: (B, V+T, D)

        return fused_feats

In [ ]:
class TextModel(nn.Module):
    def __init__(self):
        super(TextModel, self).__init__()

        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        self.bart_pho = BARTPho(device = self.device, use_lora = True)  # truyền rõ device
        self.bart_pho.freeze_encoder()
        self.encoder = BiDirectionalCrossAttention().to(self.device)

    def forward(self, ocr_infos, questions, answers):
        ocr_encoding = self.bart_pho.encode(ocr_infos)
        ocr_feats = ocr_encoding["encoder_hidden_states"]
        ocr_attention_mask = ocr_encoding["attention_mask"]
    
        question_encoding = self.bart_pho.encode(questions)
        question_feats = question_encoding["encoder_hidden_states"]
        question_attention_mask = question_encoding["attention_mask"]
    
        encoder_output = self.encoder(ocr_feats, ocr_attention_mask, question_feats, question_attention_mask)
        encoder_attention_mask = torch.cat([ocr_attention_mask, question_attention_mask], dim = 1)
    
        answer_encoded = self.bart_pho.tokenizer(
            answers, return_tensors = "pt", padding = True, truncation = True
        ).to(self.device)
        answer_input_ids = answer_encoded["input_ids"]
        answer_attention_mask = answer_encoded["attention_mask"]
    
        logits = self.bart_pho.decode(
            answer_input_ids = answer_input_ids,
            answer_attention_mask = answer_attention_mask,
            encoder_hidden_states = encoder_output,
            encoder_attention_mask = encoder_attention_mask
        )
    
        return logits

    def generate(self, ocr_infos, questions):
         # Step 1: Extract visual features
        with torch.no_grad():
            ocr_encoding = self.bart_pho.encode(ocr_infos)
            ocr_feats = ocr_encoding["encoder_hidden_states"]
            ocr_attention_mask = ocr_encoding["attention_mask"]
        
            question_encoding = self.bart_pho.encode(questions)
            question_feats = question_encoding["encoder_hidden_states"]
            question_attention_mask = question_encoding["attention_mask"]
        
            encoder_output = self.encoder(ocr_feats, ocr_attention_mask, question_feats, question_attention_mask)
            encoder_attention_mask = torch.cat([ocr_attention_mask, question_attention_mask], dim = 1)
    
            return self.bart_pho.generate(encoder_output, encoder_attention_mask)

In [ ]:
OCR_VAL_CHECKPOINT_PATH = hf_hub_download(
    repo_id = HUGGINGFACE_HUB_REPO, 
    filename = OCR_VAL_FILENAME, 
    local_dir = "./checkpoints",  # Store the checkpoint locally in the "checkpoints" directory
    local_dir_use_symlinks = False  # Avoid using symlinks for compatibility
)

print(OCR_VAL_CHECKPOINT_PATH)

val_ocr_contents = dict()
with open (OCR_VAL_CHECKPOINT_PATH, 'r', encoding = 'utf-8') as ocr_file:
    lines = ocr_file.readlines()
    for line in lines:
        temp = line.split(":")
        ocr_index = int(temp[0].split(" ")[1])
        ocr_content = ":".join(temp[1:]).strip()
        val_ocr_contents[str(ocr_index)] = ocr_content

val_ocr_contents = dict(sorted(val_ocr_contents.items(), key = lambda x: int(x[0])))

In [ ]:
OCR_TEST_CHECKPOINT_PATH = hf_hub_download(
    repo_id = HUGGINGFACE_HUB_REPO, 
    filename = OCR_TEST_FILENAME, 
    local_dir = "./checkpoints",  # Store the checkpoint locally in the "checkpoints" directory
    local_dir_use_symlinks = False  # Avoid using symlinks for compatibility
)

print(OCR_TEST_CHECKPOINT_PATH)

test_ocr_contents = dict()
with open (OCR_TEST_CHECKPOINT_PATH, 'r', encoding = 'utf-8') as ocr_file:
    lines = ocr_file.readlines()
    for line in lines:
        temp = line.split(":")
        ocr_index = int(temp[0].split(" ")[1])
        ocr_content = ":".join(temp[1:]).strip()
        test_ocr_contents[str(ocr_index)] = ocr_content

test_ocr_contents = dict(sorted(test_ocr_contents.items(), key = lambda x: int(x[0])))

In [ ]:
val_hf_dataset = load_dataset("Namronaldo2004/ViInfographicsVQA", split = "val", streaming = True)
test_hf_dataset = load_dataset("Namronaldo2004/ViInfographicsVQA", split = "test", streaming = True)
val_dataset = ViInfographicsVQATextDataset(val_hf_dataset, val_ocr_contents)
test_dataset = ViInfographicsVQATextDataset(test_hf_dataset, test_ocr_contents)

def custom_collate_fn(batch):
    ocr_infos, questions, answers = zip(*batch)
    
    ocr_infos = list(ocr_infos)
    questions = list(questions)
    answers = list(answers)
    
    return ocr_infos, questions, answers
    
val_dataloader = DataLoader(val_dataset, batch_size = 2, shuffle = False, collate_fn = custom_collate_fn)
test_dataloader = DataLoader(test_dataset, batch_size = 2, shuffle = False, collate_fn = custom_collate_fn)

In [ ]:
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

def load_checkpoint(model, filepath):
    # Load checkpoint and restore model state
    if os.path.isfile(filepath):
        checkpoint = torch.load(filepath, map_location = DEVICE)
        model.load_state_dict(checkpoint['model_state_dict'])

# Khởi tạo model
VQA_model = TextModel().to(DEVICE)
load_checkpoint(VQA_model, CHECKPOINT_PATH)
VQA_model.eval();

In [ ]:
sample = next(iter(val_hf_dataset))
ocr_info = val_ocr_contents["0"]
question = sample["question"]

VQA_model.generate([ocr_info], [question])

In [ ]:
def generate_and_save_answers(dataloader, output_file_path, dataset_name = ""):
    with open(output_file_path, 'w', encoding = 'utf-8') as f_out:
        for batch in tqdm(dataloader, desc = f"Generating answers for {dataset_name}"):
            ocr_infos, questions, _ = batch  # ignore groundtruth answers

            # Inference (generate returns list of decoded strings)
            with torch.no_grad():
                generated_answers = VQA_model.generate(ocr_infos, questions)

            # Save each generated answer to file
            for answer in generated_answers:
                f_out.write(answer.strip() + '\n')

# Generate and save for val and test
generate_and_save_answers(val_dataloader, "val_answer_flow3_text.txt", "val")
generate_and_save_answers(test_dataloader, "test_answer_flow3_text.txt", "test")